This is the main research notebook of the current project.
See full description in [README.md](https://github.com/mvulf/acm-project/blob/main/README.md)

# Import libraries and setup callbacks

In [1]:
import numpy as np

import sys
sys.path.append('../')

from regelum.simulator import CasADi

from regelum.objective import RunningObjective
from regelum.scenario import Scenario, RLScenario

# For MPC only
from regelum.utils import rg
from regelum.predictor import EulerPredictor
from regelum.model import ModelWeightContainer
# Max Iteration count may be added in opt_options={"print_level": 0, "max_iter": MAX_ITER},
from regelum.optimizable.core.configs import CasadiOptimizerConfig
from regelum.critic import CriticTrivial
from regelum.event import Event

from src.system import HydraulicSystem, StationaryHydraulicSystem
from src.observer import StationaryHydraulicObserver
from src.policy import PDController, MPC
from src.objective import HydraulicObjectiveModel


# Callbacks
from regelum.callback import ScenarioStepLogger
from src.callback import HistoricalDataCallback, SimulatorStepLogger
from regelum import set_ipython_env
%matplotlib inline

callbacks = [SimulatorStepLogger, ScenarioStepLogger, HistoricalDataCallback]
ScenarioStepLogger.cooldown = 0.01
callbacks = set_ipython_env(callbacks=callbacks, interactive=True)

# Setup common parameters and objects

In [2]:
MAX_STEP = 1e-7
FINAL_TIME = 10e-3
SAMPLING_TIME = 1e-3

# Define the initial state
p_atm = 1e5
initial_state = rg.array([1e3, 0, 0, p_atm, p_atm])

# SYSTEM and SIMULATOR
system = HydraulicSystem(
    init_state=initial_state
)
simulator = CasADi(
    system=system,
    state_init=initial_state,
    action_init=rg.array([0]),
    time_final=FINAL_TIME,
    max_step=MAX_STEP,
)

# RUNNING OBJECTIVE
running_objective_model = HydraulicObjectiveModel(
    system=system,
    quad_matrix_type="diagonal", weights=[1, 0, 0]
)  # A quadratic model for running (a.k.a. instant) objective
running_objective = RunningObjective(
    model=running_objective_model
)  # Wrap `running_objective_model` into `RunningObjective` class

# Experiments

Below you can find experiments of both PD and MPC controller

## PD-controller

In [3]:
pd_policy = PDController(
    system=system,
    sampling_time=SAMPLING_TIME,
    P_coef=25,
)

scenario = Scenario(
    policy=PDController(
        system=system,
        sampling_time=SAMPLING_TIME
    ),
    simulator=simulator,
    sampling_time=SAMPLING_TIME,
    N_episodes=1,
    N_iterations=1,
    running_objective=running_objective,
)

scenario.run()

[16:40:48] INFO     runn. objective: 3.71, state est.: [0. 0.], observation: [0. 0.], action:      ]8;id=362952;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=741487;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    [20.], value: 0.0037, time: 0.0000 (0.0%), episode: 1/1, iteration: 1/1                        

[16:40:53] INFO     runn. objective: 2.92, state est.: [  0.22 432.48], observation: [  0.22       ]8;id=205897;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=84927;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    432.48], action: [20.], value: 0.0066, time: 0.0010 (10.0%), episode: 1/1,                     
                    iteration: 1/1                                                                                 

[16:40:59] INFO     runn. objective: 1.29, state est.: [  0.79 694.23], observation: [  0.79       ]8;id=906990;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=15004;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    694.23], action: [14.67], value: 0.0079, time: 0.0020 (20.0%), episode: 1/1,                   
                    iteration: 1/1                                                                                 

[16:41:06] INFO     runn. objective: 0.17, state est.: [  1.51 738.48], observation: [  1.51       ]8;id=619009;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=408838;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    738.48], action: [5.3], value: 0.0081, time: 0.0030 (30.0%), episode: 1/1,                     
                    iteration: 1/1                                                                                 

[16:41:15] INFO     runn. objective: 0.05, state est.: [  2.15 563.61], observation: [  2.15       ]8;id=560546;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=959519;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    563.61], action: [-3.1], value: 0.0081, time: 0.0040 (40.0%), episode: 1/1,                    
                    iteration: 1/1                                                                                 

...

[16:41:39] INFO     runn. objective: 0.57, state est.: [2.68 0.3 ], observation: [2.68 0.3 ],      ]8;id=905149;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=997118;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    action: [-9.96], value: 0.0091, time: 0.0060 (60.0%), episode: 1/1, iteration:                 
                    1/1                                                                                            

[16:41:53] INFO     runn. objective: 0.57, state est.: [2.68 0.78], observation: [2.68 0.78],      ]8;id=804110;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=394448;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    action: [-9.98], value: 0.0097, time: 0.0070 (70.0%), episode: 1/1, iteration:                 
                    1/1                                                                                            

[16:42:08] INFO     runn. objective: 0.57, state est.: [2.68 0.53], observation: [2.68 0.53],      ]8;id=29290;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=499322;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    action: [-9.99], value: 0.0103, time: 0.0080 (80.0%), episode: 1/1, iteration:                 
                    1/1                                                                                            

[16:42:25] INFO     runn. objective: 0.57, state est.: [2.68 0.03], observation: [2.68 0.03],      ]8;id=151538;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=183041;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    action: [-10.], value: 0.0108, time: 0.0090 (90.0%), episode: 1/1, iteration:                  
                    1/1                                                                                            

[16:42:43] INFO     runn. objective: 0.58, state est.: [2.68 1.6 ], observation: [2.68 1.6 ],      ]8;id=858059;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=956344;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    action: [-10.02], value: 0.0114, time: 0.0100 (100.0%), episode: 1/1,                          
                    iteration: 1/1                                                                                 

In [6]:
state_trajectory_pd = callbacks[0].data

In [7]:
state_trajectory_pd

,time,jet length [mm],jet velocity [mm/s],piston position [µm],piston velocity [µm/s],throttle position [µm],hydraulic pressure [Pa],working pressure [Pa]
0,0.000000e+00,0.000000,0.000000,1000.000000,0.000000,0.000000,100000.000000,100000.000000
1,1.000000e-07,0.000000,0.000000,1000.000000,0.000000,0.001000,100000.001581,100000.000000
2,2.000000e-07,0.000000,0.000000,1000.000000,0.000000,0.002000,100000.006323,100000.000000
3,3.000000e-07,0.000000,0.000000,1000.000000,0.000000,0.003000,100000.014227,100000.000000
4,4.000000e-07,0.000000,0.000000,1000.000000,0.000000,0.004000,100000.025291,100000.000000
...,...,...,...,...,...,...,...,...
99997,9.999700e-03,2.684625,1.646984,1000.730889,0.448391,-20.256728,107046.097188,100000.002930
99998,9.999800e-03,2.684625,1.669613,1000.730889,0.454552,-20.257229,107046.037150,100000.002991
99999,9.999900e-03,2.684625,0.730293,1000.730889,0.198822,-20.257730,107045.983874,100000.000179
100000,1.000000e-02,2.684625,0.752278,1000.730889,0.204808,-20.258231,107045.926699,100000.000214
